# Configurations

In [ ]:
# These are configuration variable used by the notebook

NUBANK_FILE_PATTERN = "nubank*.csv"
SPLITWISE_FILE_PATTERN = "mozi-e-eu*.csv"
OTHER_EXPENSES_FILE_PATTERN = "Extrato outras contas*Despesas*.csv"
INCOMES_FILE_PATTERN = "Extrato outras contas*Rendimentos*.csv"

# Directory that contains the expenses data
DATA_PATH = "./data"

# Person name that the expenses will be extracted from Splitwise csv
PERSON_WHO_PAYS = "Lucas Alencar"

# Category Map from Nubank/Splitwise to official categories
# This map will be used to convert a category to another one
CATEGORY_CONVERSION_TABLE = {
    'Eletricidade': 'Casa',
    'Aluguel': 'Casa',
    'Móveis': 'Casa',
    'Jantar fora': 'Restaurante',
    'Supermercado': 'Mercado',
    'Eletrônicos': 'Compras',
    'Vestuário': 'Compras',
    'Produtos de limpeza': 'Compras',
    'Filmes': 'Lazer',
    'Música': 'Lazer',
    'TV/Telefone/Internet': 'Serviços',
    'Despesas médicas': 'Saúde',
    'Táxi': 'Transporte',
    'Ônibus/trem': 'Viagem',
    'Avião': 'Viagem',
    'Hotel': 'Viagem',
    'Presentes': 'Outros',
    'Geral': 'Outros',
    'Ajuste': 'Outros',
    'Manutenção': 'Compras'
}

from datetime import date
from date_helpers import previous_month

# Date used to make month analysis through the notebook
# BASE_DATE = date(2018, 7, 1)
BASE_DATE = previous_month(date.today())

# Year you were born
YEAR_OF_BIRTH = 1991

# Data load and preprocess

In [ ]:
from preprocess import *

# NUBANK PREPROCESS
nubank_expenses = nubank_preprocess(read_all_csvs(DATA_PATH, NUBANK_FILE_PATTERN))

# SPLITWISE PREPROCESS
splitwise = read_first_file_found(DATA_PATH, SPLITWISE_FILE_PATTERN)
splitwise = splitwise_preprocess(splitwise, PERSON_WHO_PAYS)

# OTHER EXPENSES PREPROCESS
other_expenses = read_first_file_found(DATA_PATH, OTHER_EXPENSES_FILE_PATTERN)

# INCOMES PREPROCESS
incomes = read_first_file_found(DATA_PATH, INCOMES_FILE_PATTERN)
incomes = incomes_preprocess(incomes)

# Data consolidation and exports
expenses_input_files = [nubank_expenses, splitwise, other_expenses]
expenses = expenses_preprocess(pd.concat(expenses_input_files, sort=False), CATEGORY_CONVERSION_TABLE)

export_to_sheets(expenses, DATA_PATH)
export_raw_expenses(expenses, DATA_PATH)

# Monthly Expenses

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

%matplotlib inline

from IPython.display import display
from monthly_expenses import *
from record_summary import total_amount_by, groupby_month

MONTHLY_BALANCE_GOAL = 0.4

# Expenses and incomes by month
exp_by_month = total_amount_by(groupby_month(expenses), expenses)

income_salary = incomes[incomes.category == 'renda'][['date', 'amount']]
inc_by_month = total_amount_by(groupby_month(income_salary), income_salary)

monthly_exp = summary_expenses(exp_by_month, inc_by_month)

# Monthly averages
avg_monthly_exp = summary_expenses(exp_by_month.mean(), inc_by_month.mean())
avg_monthly_exp.index = ['Average monthly expenses']

print(" >>> Monthly balance goal:", MONTHLY_BALANCE_GOAL * 100, "%")
display(style_summary_expenses(monthly_exp, MONTHLY_BALANCE_GOAL))
display(style_summary_expenses(avg_monthly_exp, MONTHLY_BALANCE_GOAL))

# Plot expenses summary
plt.figure(figsize=(20,10))
plt.plot(monthly_exp['Expenses'] * -1, 'r', label='Expenses')
plt.plot(monthly_exp['Incomes'], 'g', label='Incomes')
plt.plot(monthly_exp['Balance'], 'b', label='Balance')
plt.legend(fontsize=20)
plt.grid()
plt.show()

# Expenses by category

In [ ]:
from date_helpers import records_for_month

from monthly_expenses import *

# BASE_DATE = date(2018, 10, 1)
print(">>> Month of analysis: ", BASE_DATE.month, BASE_DATE.year)

expenses_for_month = records_for_month(expenses, BASE_DATE)
incomes_for_month = records_for_month(incomes, BASE_DATE)

expenses_for_category_by_total = describe_expenses(expenses_for_month, incomes_for_month)
display(style_expenses_distribution(expenses_for_category_by_total))

def display_expenses_by_category(category):
    print("Expenses in category", category)
    display(expenses_for_month[expenses_for_month.category == category].sort_values('amount'))
    
display_expenses_by_category("outros")

# Plots over time for each category grouping them by easy of read
expenses_over_time = expenses_over_time(expenses, incomes, 'amount #')

plot_expenses_over_time(expenses_over_time[['casa', 'viagem', 'educação']])
plot_expenses_over_time(expenses_over_time[['restaurante', 'mercado', 'serviços']])
plot_expenses_over_time(expenses_over_time[['transporte', 'lazer']])

# Investments progress

In [ ]:
from IPython.display import display
from monthly_investiments import *
from central_bank_data import central_bank_metric, BC_IPCA_BY_MONTH_ID


# BASE_DATE = date(2018, 10, 1)
print(">>> Month of analysis:", BASE_DATE.month, BASE_DATE.year)

current_ipca = central_bank_metric(BC_IPCA_BY_MONTH_ID, BASE_DATE)
current_ipca = current_ipca * 100 if current_ipca else None
print("> Current IPCA:", current_ipca, "%")

RETURN_FOR_MONTH_GOAL = 0.005
RETURN_WITH_INFLATION_GOAL = 0.005

invest = incomes[incomes.category.isin(['aplicação', 'valor aplicado'])]

summary_invest = summary_investiments_current_month(invest, BASE_DATE)\
                 .sort_values('Return for month (%)', ascending=False)
display(style_summary_investments(summary_invest, RETURN_FOR_MONTH_GOAL, RETURN_WITH_INFLATION_GOAL))

assets_summary = summary_assets(invest)
display(style_summary_assets(assets_summary))

plot_invest_type_distribution(invest, BASE_DATE)

# Plot line to see assets progress
plot_assets_summary(assets_summary[['Return']], '2018-02')
plot_assets_summary(assets_summary[['Applications']], '2018-02')

# Assets goals

In [ ]:
# PMS Patrimônio Mínimo de Sobrevivência (Reserva de emergência)
# 6 * Custo mensal

# PMR Patrimônio Mínimo Recomendado para segurança 
# Possibilita decisões mais arriscada sobre carreira ou mudanças.
# 12 * Custo mensal

# PI Patrimônio Ideal para idade e consumo 
# Dado os meus gastos mensais atuais, se eu continuar trabalhando até a aposentadoria (65 anos), 
# eu conseguiria viver apenas gastando esse dinheiro.
# 0.1 * PMR * Idade

# PNIF Patrimônio Necessário para a Independência Financeira
# Dado os meus rendimentos com investimentos, eu conseguiria manter o mesmo padrão apenas com o dinheiro investido
# PMR / Rendimento anual %

from datetime import date
from monthly_investiments import *

age = date.today().year - YEAR_OF_BIRTH

# This is the annualized return based on the return for every month
# It is starting to count the return beginning on 2018-04, 
# because this is the most accurate values I have now
annualized_return = assets_summary.loc['2018-04':, 'Return / Total'].mean() * 12
print("Annualized return:", annualized_return * 100, '%')

pms = 6 * exp_by_month.mean() * -1
pmr = 12 * exp_by_month.mean() * -1
pi = 0.1 * pmr * age
pnif = pmr / annualized_return
assets_total = invested_for_month_by(invest, BASE_DATE, 'title').sum()

assets_goals = pd.DataFrame([pms, pmr, pi, pnif, assets_total], index=['PMS', 'PMR', 'PI', 'PNIF', 'Total'])
display(style_assets_goals(assets_goals, assets_total.amount))

plot_assets_summary(assets_summary[['Total']], '2018-01')

# How much my time is worth?

In [ ]:
from date_helpers import records_for_month
import calendar

def weekdays_in_month(BASE_DATE):
    month_calendar = calendar.monthcalendar(BASE_DATE.year, BASE_DATE.month)
    weekdays_in_month = 0
    for week in month_calendar:
        for day in filter(lambda x: x > 0, week):
            if calendar.weekday(BASE_DATE.year, BASE_DATE.month, day) < 5:
                weekdays_in_month += 1
    return weekdays_in_month

# BASE_DATE = date(2018, 6, 1)

total_earned_on_base_date = records_for_month(incomes[incomes.category == 'renda'], BASE_DATE).amount.sum()
total_worked_by_day = 8.8 # According to CLT
total_worked_by_month = total_worked_by_day * weekdays_in_month(BASE_DATE)

money_for_time = total_earned_on_base_date / total_worked_by_month

print("In", BASE_DATE.strftime('%Y-%m'), "you have earned", total_earned_on_base_date, ".")
print("Given you have worked for {:,.2f} hours in this month.".format(total_worked_by_month))
print("Your time is worth {:,.2f} R$/hour.".format(money_for_time), "Not considering holidays or vacations.")

# TODO

- Categorize my investments in patrimony goals (SELIC goes inside Emergency Reserve, while risky investments go to Financial Independence)
- Add index to measure my financial independence
- Think about another ways of seeing investments return for month